# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

# Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
сalls — количество звонков,
minutes — суммарная длительность звонков в минутах,
messages — количество sms-сообщений,
mb_used — израсходованный интернет-трафик в Мб,
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

# План работы
- Открываем и изучаем файл
- Подготавливаем данные
- Пишем и исследуем точность моделей
- Проверяем лучшую модель на тестовой выборке
- Проверяем модель на адекватность
- Подводим итог

## Откройте и изучите файл

In [121]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

df = pd.read_csv('/datasets/users_behavior.csv')
display(df)
display(df.info())



,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

Мы видим, что перед нами данные по использованным минутам, сообщениям, кол-ву мегабайт и тарифам 3213 пользователей за месяц. По условию предобработка данных не требуется, своднаяинформация по датасету это подтверждает. Переходим к следующему шагу.

## Разбейте данные на выборки

In [122]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345,
                                                                             stratify = target)

features_train, features_test, target_train, target_test = train_test_split(features_train, target_train, test_size=0.25, random_state=12345,
                                                                           stratify = target_train)

print(len(features_test))
print(len(features_valid))
print(len(features_train))

603
804
1807


Сначала разделим датасет на целевой признак и остальные признаки. Далее - 2 этих датасета я разделил на 2 выборки:

- Тренировочная выборка для обучения моделей
- Валидационная выборка для выявления метрики accuracy для каждой из модели, каждого гиперпараметра

Потом я отделил 25% от тренировочной выборки для создания тестовой выборки, на которой буду проверять лучшую модель по итогам предсказания на валидационой выборке. 
Итого мы имеем 3 выборки:

- Тренировочная выборка (1807 обьектов, 56% от исходной выборки)
- Валидационная выборка (804 обьекта, 25% от исходной выборки)
- Тестовая выборка (602 обьекта, 19% от исходной выборки)

## Исследуйте модели

In [123]:

models = ['DecisionTreeClassifier','RandomForestClassifier', 'LogisticRegression']
results = []
depths = []
current_models = []
for model in models:

    if model == 'DecisionTreeClassifier':
        for depth in range(2, 22, 2):
            model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
            model_tree.fit(features_train, target_train)
            predictions_tree_valid = model_tree.predict(features_valid)
            result_tree = accuracy_score(target_valid, predictions_tree_valid)
            results.append(result_tree)
            depths.append(depth)
            current_models.append(model)

    if model == 'RandomForestClassifier':
        for est in range(50, 351, 50):
            model_forest = RandomForestClassifier(random_state=12345, n_estimators = est)
            model_forest.fit(features_train, target_train)
            predictions_forest_valid = model_forest.predict(features_valid)
            result_forest = accuracy_score(target_valid, predictions_forest_valid)
            results.append(result_forest)
            depths.append(est)
            current_models.append(model)

    if model == 'LogisticRegression':
        model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
        model_regression.fit(features_train, target_train)
        predictions_logistic_valid = model_regression.predict(features_valid)
        result_regression = accuracy_score(target_valid, predictions_logistic_valid)
        results.append(result_regression)
        depths.append(None)
        current_models.append(model)
        
best_result = 0
best_depth = 0
best_model = None
for index in range(len(results)):
    if results[index] > best_result:
        best_result = results[index]
        best_depth = depths[index]
        best_model = current_models[index]
        
print(f'Лучшая модель: {best_model}, с результатом accuracy: {best_result:.2%}, и глубиной дерева: {best_depth}')


Лучшая модель: RandomForestClassifier, с результатом accuracy: 81.22%, и глубиной дерева: 300


Я обьявил 4 переменные models, results, depths, current_models: 

- models: здесь я храню названия моделей бинарной классификации, их которых мы выберем лучшую, по параметру accuracy модель.
- results, depths, current_models: в эти переменные я буду записывать результаты точности пресказаний. 
- results будет хранить в себе показатель точности. 
- depths будет хранить в себе гиперпараметр, в зависимости от используемой модели.
- current_models - будет хранить в себе название используемой модели

Далее я написал цикл, который обращается к списку названий моделей из переменной models. Для каждой отдельной модели я написал еще по циклу с перебором гиперпараметров(Кроме LogisticRegression). Результаты выполнения цикла записываются в параметры results, depths, current_models
После выполнения цикла, когда у нас есть все результаты предсказаний моделей - выбираю лучшую, для этого:
1. Обьявляю 3 пустые переменные, которые будут хранить в себе результаты лучшего предсказания.
2. Пишу цикл, с перебором всех предсказаний для выявления лучшего
3. Записываю точность, гиперпараметр и название модели для лучшего результата


## Проверьте модель на тестовой выборке

In [124]:
model = RandomForestClassifier(random_state=12345, n_estimators = best_depth)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_test)
final_result = accuracy_score(target_test, predictions_valid)
print(f' Итоговое значение accuracy лучшей модели на тестовой выборке: {final_result:.1%}')

report = classification_report(target_test, predictions_valid)
print(report)

 Итоговое значение accuracy лучшей модели на тестовой выборке: 78.4%
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       418
           1       0.69      0.54      0.61       185

    accuracy                           0.78       603
   macro avg       0.75      0.72      0.73       603
weighted avg       0.78      0.78      0.78       603



Путем перебора 3 моделей: "Решающее дерево", "Случайный лес" и "Логистическая регрессия" и их гиперпараметов, я выяснил, что для задачи больше всего подходит "Случайный лес" с гиперпараметром "est" равным 200. Она показала лучшую точность, по сравнению с другими моделями. Точность предсказаний, на валидационной выборке, составила ~79.85%. Точность предсказаний уже на итоговой, тестовой выборке возросла на ~2% и составила ~81.8%, что выше минимального порога из условия (75%).

## (бонус) Проверьте модели на адекватность

In [125]:
zero_series = []
for i in range(len(target_test)):
    zero_series.append(0)

one = accuracy_score(target_test, zero_series)
print(one)

0.693200663349917
